<a href="https://colab.research.google.com/github/NSoni25/COMP8240_Major-_Project/blob/main/COMP8240_Major_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abstractive Summaries of online articles

Installing Pytorch, sentencepiece, rougescore and Hugging Face Transformer.

In [ ]:
!pip3 install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |█████████████▌                  | 834.1 MB 1.4 MB/s eta 0:14:03tcmalloc: large alloc 1147494400 bytes == 0x55fa53db2000 @  0x7f1b3f6a4615 0x55fa521c14cc 0x55fa522a147a 0x55fa521c42ed 0x55fa522b5e1d 0x55fa52237e99 0x55fa522329ee 0x55fa521c5bda 0x55fa52237d00 0x55fa522329ee 0x55fa521c5bda 0x55fa52234737 0x55fa522b6c66 0x55fa52233daf 0x55fa522b6c66 0x55fa52233daf 0x55fa522b6c66 0x55fa52233daf 0x55fa521c6039 0x55fa52209409 0x55fa521c4c52 0x55fa52237c25 0x55fa522329ee 0x55fa521c5bda 0x55fa52234737 0x55fa522329ee 0x55fa521c5bda 0x55fa52233915 0x55fa521c5afa 0x55fa52233c0d 0x55fa522329ee
     |█████████████████               | 1055.7 MB 1.6 MB/s eta 0:09:33tcmalloc: large alloc 1434370048 bytes == 0x55fa98408000 @  0x7f1b3f6a4615 0x55fa521c14cc 0x55fa522a147a 0x55fa521c42ed 0x55fa522b5e1d 0x55fa52237e99 0x55fa522329ee 0x55fa521c5bda 0x55fa52237d00 0x55fa522329ee 0x55fa521c5bda 0x55fa52234737 0x55fa522b6c66 0x55fa5

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 11.2 MB/s 
     |████████████████████████████████| 596 kB 75.9 MB/s 
     |████████████████████████████████| 895 kB 70.3 MB/s 
     |████████████████████████████████| 3.3 MB 80.5 MB/s 
     |████████████████████████████████| 59 kB 4.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install rouge_score

In [ ]:
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 11.0 MB/s 


Importing libraries for generating abstractive summaries

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import requests
from bs4 import BeautifulSoup
import re
import torch
from rouge_score import rouge_scorer

In [ ]:
#Model intialization
model_name = "google/pegasus-reddit_tifu"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

In [ ]:
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [ ]:
tokenizer

PreTrainedTokenizer(name_or_path='google/pegasus-reddit_tifu', vocab_size=96103, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask_2>', 'additional_special_tokens': ['<mask_1>', '<unk_2>', '<unk_3>', '<unk_4>', '<unk_5>', '<unk_6>', '<unk_7>', '<unk_8>', '<unk_9>', '<unk_10>', '<unk_11>', '<unk_12>', '<unk_13>', '<unk_14>', '<unk_15>', '<unk_16>', '<unk_17>', '<unk_18>', '<unk_19>', '<unk_20>', '<unk_21>', '<unk_22>', '<unk_23>', '<unk_24>', '<unk_25>', '<unk_26>', '<unk_27>', '<unk_28>', '<unk_29>', '<unk_30>', '<unk_31>', '<unk_32>', '<unk_33>', '<unk_34>', '<unk_35>', '<unk_36>', '<unk_37>', '<unk_38>', '<unk_39>', '<unk_40>', '<unk_41>', '<unk_42>', '<unk_43>', '<unk_44>', '<unk_45>', '<unk_46>', '<unk_47>', '<unk_48>', '<unk_49>', '<unk_50>', '<unk_51>', '<unk_52>', '<unk_53>', '<unk_54>', '<unk_55>', '<unk_56>', '<unk_57>', '<unk_58>', '<unk_59>', '<unk_60>', '<unk_61>', '<

In [ ]:
model_dic = []
mod = 'Reddit_tifu'
data_souce = 'articles'
blog = ["https://datascience.codata.org/articles/10.5334/dsj-2021-009/",
        "https://datascience.codata.org/articles/10.5334/dsj-2021-027/",
       "https://datascience.codata.org/articles/10.5334/dsj-2021-026/",
       "https://datascience.codata.org/articles/10.5334/dsj-2021-025/",
       "https://datascience.codata.org/articles/10.5334/dsj-2021-024/",
       "https://datascience.codata.org/articles/10.5334/dsj-2020-015/",
       "https://datascience.codata.org/articles/10.5334/dsj-2020-013/",
       "https://datascience.codata.org/articles/10.5334/dsj-2021-028/",
        "https://datascience.codata.org/articles/10.5334/dsj-2015-002/",
        "https://datascience.codata.org/articles/10.5334/dsj-2015-011/",
        "https://datascience.codata.org/articles/10.5334/dsj-2021-032/",
        "https://datascience.codata.org/articles/10.5334/dsj-2019-055/"]



In [ ]:
#Function to scrape PDF and generate abstractive of them
def getdata(url):
  text = []
  r = requests.get(url)
  d=""
  s = BeautifulSoup(r.content, 'html.parser')
  data = ''
  title = []
  summary = []
  for data in s.find_all('p'):
    d+=data.get_text()
    d = re.sub(r'https?:\/\/\S*', ' ',d, flags = re.MULTILINE)
    d= re.sub('[^a-zA-Z0-9]', ' ', d)
    d=  re.sub(r"[0-9]", "", d)
    d = d.lower()
    d=' '.join(d.split())
  input = tokenizer(d, truncation=True, padding='longest', return_tensors="pt").to(device)
  translated = model.generate(**input)
  summary = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return summary

In [ ]:
for i in blog:
  model_dic.append(getdata(i))

In [ ]:
model_dic

[['data management is fundamental to ensuring reproducible open scientific research however sufficient research data assistance is often not offered at universities and when offered typically only provides basic services that are viewed as optional integrating information specialists into research groups provides a potentially promising means of improving data management by providing personalized data management workflows workflows should include a file organization scheme the creation of data management roles for members a data storage sharing guide and training and evaluation librarians who regularly interact with faculty and students and are familiar with data management tools are uniquely situated to assist with the creation and assessment of these workflowsorganizing and sharing research data is a fundamental part of the research process mons that has been shown'],
 ['collection and open sharing of wastewater based epidemic data potentially provide immense public health benefits d

The above output is the predicted summary generated by our function getdataurl of 12 articles applied as a input. The text extracted from the online articles were first clean and then pass as a input to the model for abstractive summary generation.

In [ ]:
#Function that scrape only abstracts so that it can be used as original summary
def original_summary(url):
  r = requests.get(url)
  sum = []
  text = []
  d=""
  soup = BeautifulSoup(r.content, 'html.parser')
  for data in soup.find_all("p")[1]:
    d = re.sub(r'https?:\/\/\S*', "",str(data))
    d= re.sub(r'[^a-zA-Z0-9]', " ", d)
    d=  re.sub(r"[0-9]", " ",d)
    d = d.lower()
    d=' '.join(d.split())
    sum.append(d)
  return(sum)

In [ ]:
originalsummary = []
summary = []
for i in blog:
  summary+=original_summary(i)

In [ ]:
print(summary)

['comprehensive research data management is fundamental to ensuring reproducible open scientific research however sufficient research data assistance is often not offered at universities and when offered typically only provides basic services that are viewed as optional integrating information specialists into research groups provides a potentially promising means of improving data management by providing personalized data management workflows workflows are comprehensive executable guides that require planning implementation feedback and adaptation comprehensive data management workflows should include a file organization scheme the creation of data management roles for members a data storage sharing guide and training and evaluation librarians who regularly interact with faculty and students and are familiar with data management tools are uniquely situated to assist with the creation and assessment of these workflows', 'collection and open sharing of wastewater based epidemic data pot

The above output is the original summary of the online articles that were extracted using original_function and stored in summary list

In [ ]:
#Function that calculate score all the summaries 
def rougescorer(b):
  precision= []
  recall=[]
  fmeasure=[]
  a=[]
  b=[]
  c=[]
  colnames = ['precision','recall','fmeasure']
  df = pd.DataFrame(columns =colnames)
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
  for b in range(0,12):
    scores = scorer.score(str(summary[b]),str(model_dic[b]))
    a=(scores['rouge1'].precision)
    b=(scores['rouge1'].recall)
    c=(scores['rouge1'].fmeasure)
    precision.append(a)
    recall.append(b)
    fmeasure.append(c)
  df['precision'] = precision
  df['recall'] = recall
  df['fmeasure'] = fmeasure
  return df

In [ ]:
import pandas as pd
for j in blog:
  eval_score=rougescorer(j)

In [ ]:
print(eval_score)

    precision    recall  fmeasure
0    0.887097  0.887097  0.887097
1    1.000000  0.668478  0.801303
2    0.844828  0.632258  0.723247
3    0.907407  0.980000  0.942308
4    1.000000  0.596859  0.747541
5    0.000000  0.000000  0.000000
6    0.500000  0.983333  0.662921
7    0.687500  0.865169  0.766169
8    0.991870  0.802632  0.887273
9    0.991525  0.879699  0.932271
10   0.410526  0.458824  0.433333
11   1.000000  0.873239  0.932331


In [ ]:
eval_score

,precision,recall,fmeasure
0,0.887097,0.887097,0.887097
1,1.000000,0.668478,0.801303
2,0.844828,0.632258,0.723247
3,0.907407,0.980000,0.942308
4,1.000000,0.596859,0.747541
5,0.000000,0.000000,0.000000
6,0.500000,0.983333,0.662921
7,0.687500,0.865169,0.766169
8,0.991870,0.802632,0.887273
9,0.991525,0.879699,0.932271


In [32]:
column_names = ["model", "data_source", "humansummary", "predicted_summary","roughscorer","precision","recall","fmeasure"]
reddit_model_eval = pd.DataFrame(columns = column_names)

In [33]:
mod = 'Reddit_tifu'
data_source = 'articles'
reddit_model_eval['humansummary'] =summary
reddit_model_eval['model'] = mod
reddit_model_eval['data_source'] = data_source
reddit_model_eval['predicted_summary'] =model_dic
roughscore = 'rouge1'
reddit_model_eval['roughscorer'] = roughscore
reddit_model_eval['precision'] = eval_score.precision
reddit_model_eval['recall'] = eval_score.recall
reddit_model_eval['fmeasure'] = eval_score.fmeasure

In [34]:
reddit_model_eval

,model,data_source,humansummary,predicted_summary,roughscorer,precision,recall,fmeasure
0,Reddit_tifu,articles,comprehensive research data management is fund...,[data management is fundamental to ensuring re...,rouge1,0.887097,0.887097,0.887097
1,Reddit_tifu,articles,collection and open sharing of wastewater base...,[collection and open sharing of wastewater bas...,rouge1,1.000000,0.668478,0.801303
2,Reddit_tifu,articles,principal component analysis pca is a commonly...,[in certain classification problems even extre...,rouge1,0.844828,0.632258,0.723247
3,Reddit_tifu,articles,citizen science cs projects are part of a new ...,[citizen science cs projects are part of a new...,rouge1,0.907407,0.980000,0.942308
4,Reddit_tifu,articles,the southern african science service centre fo...,[sasscal was initiated to support regional wea...,rouge1,1.000000,0.596859,0.747541
5,Reddit_tifu,articles,some of the early research data alliance worki...,[rda rda rda rda rda rda rda rda rda rda rda r...,rouge1,0.000000,0.000000,0.000000
6,Reddit_tifu,articles,the rosa project is focused on the investigati...,[rosa project is focused on the investigations...,rouge1,0.500000,0.983333,0.662921
7,Reddit_tifu,articles,open access free access and the public domain ...,[the international nucleotide sequence databas...,rouge1,0.687500,0.865169,0.766169
8,Reddit_tifu,articles,high quality data are the precondition for ana...,[high quality data are the prerequisite for an...,rouge1,0.991870,0.802632,0.887273
9,Reddit_tifu,articles,the fields of astrostatistics and astroinforma...,[astrostatistics and astroinformatics are vita...,rouge1,0.991525,0.879699,0.932271


In the above output, I compare the predicted summary with the human summary, here human summary is the abstract of articles. So, I calculate the score by applying the scorer function on the column human summary and predicted_summary and obtained high value of Fmeasure. For e.g. for the article 11, value of Fmeasure is 0.932 which clearly states that our predicted summary and human summary are very much similar to each other, similarly for the article 10, Fmeasure is 0.43 which shows that human summary and predicted are not similar to each other. There is also a case where our model was not able to predict summary, article 5 shows that model was unable to predict summary for that, so its precision, recall and Fmeasure are all zero.

In [35]:
human_evaluation = [4,3,3,5,3,1,3,4,4,5,2,5]
reddit_model_eval['human_evaluation'] = human_evaluation

In [36]:
reddit_model_eval

,model,data_source,humansummary,predicted_summary,roughscorer,precision,recall,fmeasure,human_evaluation
0,Reddit_tifu,articles,comprehensive research data management is fund...,[data management is fundamental to ensuring re...,rouge1,0.887097,0.887097,0.887097,4
1,Reddit_tifu,articles,collection and open sharing of wastewater base...,[collection and open sharing of wastewater bas...,rouge1,1.000000,0.668478,0.801303,3
2,Reddit_tifu,articles,principal component analysis pca is a commonly...,[in certain classification problems even extre...,rouge1,0.844828,0.632258,0.723247,3
3,Reddit_tifu,articles,citizen science cs projects are part of a new ...,[citizen science cs projects are part of a new...,rouge1,0.907407,0.980000,0.942308,5
4,Reddit_tifu,articles,the southern african science service centre fo...,[sasscal was initiated to support regional wea...,rouge1,1.000000,0.596859,0.747541,3
5,Reddit_tifu,articles,some of the early research data alliance worki...,[rda rda rda rda rda rda rda rda rda rda rda r...,rouge1,0.000000,0.000000,0.000000,1
6,Reddit_tifu,articles,the rosa project is focused on the investigati...,[rosa project is focused on the investigations...,rouge1,0.500000,0.983333,0.662921,3
7,Reddit_tifu,articles,open access free access and the public domain ...,[the international nucleotide sequence databas...,rouge1,0.687500,0.865169,0.766169,4
8,Reddit_tifu,articles,high quality data are the precondition for ana...,[high quality data are the prerequisite for an...,rouge1,0.991870,0.802632,0.887273,4
9,Reddit_tifu,articles,the fields of astrostatistics and astroinforma...,[astrostatistics and astroinformatics are vita...,rouge1,0.991525,0.879699,0.932271,5


In the above output, Human evaluation is done based on comparing predict summary with the human summary column. Value 1 in human evalution column represent poor summary, similarly value 5 in human evalution represents Great summary.

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
with open('/gdrive/My Drive/reddit_tifu.csv', 'w') as f:
  reddit_model_eval.to_csv(f)